# Example of using plotlyviz for visualizing Kepler Mapper graphs

In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
import sklearn
import kmapper as km
from kmapper.plotlyviz import plotlyviz

import plotly.graph_objs as go
import ipywidgets as ipw

In [6]:
data = np.genfromtxt('cat-reference.csv', delimiter=',')

Transform  data points to get a cat in normal position:

In [7]:
x = np.copy(data[:, 0])
y = np.copy(data[:, 1])
z = np.copy(data[:,2])
data[:, 0] = z
data[:, 1] = x
data[:, 2] = y

In [8]:
axis = dict(showbackground=True, 
            backgroundcolor="rgb(230, 230,230)",
            gridcolor="rgb(255, 255, 255)",      
            zerolinecolor="rgb(255, 255, 255)")

cat_trace = dict(type='scatter3d',
                 x=data[:,0],
                 y=data[:,1],
                 z=data[:,2],
                 mode='markers',
                 marker=dict(color='grey', size=1.5))

cat_layout=layout=dict(width=600, height=600, autosize=False,
                       title='Cat',
                       scene=dict(xaxis=axis,
                                  yaxis=axis, 
                                  zaxis=axis, 
                                  
                                  aspectratio=dict(x=1.25,
                                                   y=0.4, 
                                                   z=0.8)),
                       margin=dict(r=10, b=10)
                      )
fwcat=go.FigureWidget(data=[cat_trace], layout=cat_layout)  
fwcat

FigureWidget({
    'data': [{'marker': {'color': 'grey', 'size': 1.5},
              'mode': 'markers',
      …

We can define a custom Plotly colorscale for graph node coloring, or use the default.

In [9]:
pl_matter = [[0.0, 'rgb(253, 237, 176)'], #derived from cmocean.cm.matter https://matplotlib.org/cmocean/
             [0.1, 'rgb(250, 202, 143)'],
             [0.2, 'rgb(245, 166, 114)'],
             [0.3, 'rgb(238, 132, 93)'],
             [0.4, 'rgb(226, 97, 82)'],
             [0.5, 'rgb(206, 67, 86)'],
             [0.6, 'rgb(179, 46, 94)'],
             [0.7, 'rgb(147, 31, 99)'],
             [0.8, 'rgb(114, 25, 95)'],
             [0.9, 'rgb(79, 21, 82)'],
             [1.0, 'rgb(47, 15, 61)']]

In [10]:
mapper = km.KeplerMapper(verbose=0)
lens = mapper.fit_transform(data)
scomplex = mapper.map(lens,
                      data,
                      clusterer=sklearn.cluster.DBSCAN(eps=0.1, min_samples=5),
                      coverer=km.Cover(nr_cubes=15, overlap_perc=0.2))

In [11]:
title = 'Topological network associated to<br> the cat dataset'

plotlyviz(scomplex, colorscale=pl_matter, title=title)

FigureWidget({
    'data': [{'hoverinfo': 'none',
              'line': {'color': 'rgb(180,180,180)', 'width':…

To display the kmapper graph,  just type its figure name, `fw_graph`, in the next cell:

To display more info on the generated kmapper-graph, define two more FigureWidget(s):  
the global node distribution figure, and a dummy figure
that displays info on the  algorithms involved in getting the graph from data, as well as  sklearn  class instances.

A FigureWidget has event listeners for hovering, clicking or selecting. Using the first one for `fw_graph`
we   define, via the function `hovering_widgets()`, widgets that display the node distribution, when the node is hovered over, and two textboxes for the cluster size and the member ids/labels of the hovered node members:

In [ ]:
plotlyviz(scomplex, title=title, dashboard=True)

The default widths for the  summary and histogram figures are chosen such that their sums to be at most 1000px (=600+400).
When we call the function `summary_fig()` we have to set only its height (eventually by trial and error).

In [ ]:
plotlyviz(scomplex, colorscale=pl_matter, title=title, vbox=True)

Hover the graph nodes in the next cell to see their  distribution in the right figure. Under the graphs notice the contents of two text widgets:
    the cluster size, repsctively the member ids or their labels.

Finally, each FigureWidget can be saved as a png, pdf, svg image file, as well as eps file (the eps files cannot be
defined/saved under the  Windows OS).

In [ ]:
#import plotly.io as pio
#pio.write_image(fw_graph, 'mapper-cat.pdf')#or 'mapper-cat.png'